# Importing packages and installing openai

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.0 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.54.5
    Uninstalling openai-1.54.5:
      Successfully uninstalled openai-1.54.5


### You should change ```"your-key"``` with your key from <a>https://platform.openai.com/api-keys</a>

In [6]:
key = "your-key"

openai.api_key = key

### First test
To check if it works and see what information we get.

In [11]:
import openai
import math
import numpy as np
import json

# Effectuer une requête avec l'API de complétion pour obtenir les logprobs
response = openai.Completion.create(
    model="gpt-3.5-turbo-instruct",  # Ou "gpt-4" si vous y avez accès
    prompt="Le chat mange...",  # Votre texte d'entrée
    max_tokens=25,  # Nombre maximal de tokens pour la réponse
    logprobs=5,  # Demander les logprobs pour les 5 meilleurs tokens
    top_p=1,
    presence_penalty=0.5,
    frequency_penalty=0.5,
    temperature=0.1
)

print(response['choices'][0]["text"], end="\n\n")
print([i for i in response['choices'][0]])
print([i for i in response['choices'][0]["logprobs"]])
print(response['choices'][0]["logprobs"]["tokens"])
print([round(np.exp(i),3) for i in response['choices'][0]["logprobs"]["token_logprobs"]])


de la nourriture.

['text', 'index', 'logprobs', 'finish_reason']
['tokens', 'token_logprobs', 'top_logprobs', 'text_offset']
['\n', 'de', ' la', ' nour', 'rit', 'ure', '.']
[0.234, 0.382, 0.998, 0.986, 1.0, 1.0, 0.762]


### What we can get ?
So, we can get :
1. The text > The actual output
2. the logprobs
  1. The tokens (of the output)
  2. their corresponding logprobs
  3. top other tokens with their respective logprobs

In [13]:
# Les autres suggestions
top_five = response['choices'][0]["logprobs"]["top_logprobs"]
for k in range(len(top_five)) :
  print('Suggestion n°'+str(k)+"\t", *[(i[0], round(np.exp(i[1]),3)) for i in top_five[k].items()], sep=",\t", end="\n\n")

Suggestion n°0	,	('\n\n', 0.256),	('\n', 0.234),	(' des', 0.035),	(' du', 0.019),	(' un', 0.018)

Suggestion n°1	,	('de', 0.382),	('des', 0.14),	('La', 0.108),	('...', 0.075),	('du', 0.073)

Suggestion n°2	,	(' la', 0.998),	(' tout', 0.001),	(' l', 0.0),	(' cro', 0.0),	(' sour', 0.0)

Suggestion n°3	,	(' nour', 0.986),	(' vi', 0.01),	(' p', 0.001),	(' sour', 0.001),	(' cro', 0.0)

Suggestion n°4	,	('rit', 1.0),	('r', 0.0),	('ri', 0.0),	('iture', 0.0),	('<|endoftext|>', 0.0)

Suggestion n°5	,	('ure', 1.0),	('ur', 0.0),	('u', 0.0),	('<|endoftext|>', 0.0),	('ue', 0.0)

Suggestion n°6	,	('.', 0.762),	(' ou', 0.148),	(',', 0.067),	('.\n', 0.012),	('<|endoftext|>', 0.004)



## Asking for a prompt, and output the answer

In [ ]:
x = str(input("prompt ?\n\n"))
response = openai.Completion.create(
    model="gpt-3.5-turbo-instruct",  # Ou "davinci-002"
    prompt=x,  # Votre texte d'entrée
    max_tokens=50,  # Nombre maximal de tokens pour la réponse
    logprobs=5,  # Demander les logprobs pour les 5 meilleurs tokens
    top_p=1,
    presence_penalty=0.5,
    frequency_penalty=0.5
)
print(response['choices'][0]["text"])

prompt ?

Le chat mange ...


de la nourriture. 


## Stock objects : tokens and probabilities
Here we create objects, tokens is a list of the output tokens, logprobs is the file of their corresponding log probabilities, probs are the probabilities

In [ ]:
tokens = response['choices'][0]["logprobs"]["tokens"]

logprobs = response['choices'][0]["logprobs"]["token_logprobs"]

probs = [np.exp(i) for i in logprobs]

print(*[(j, round(probs[i],2)) for i, j in enumerate(tokens)], sep="\n")

('\n\n', 0.37)
('de', 0.61)
(' la', 1.0)
(' nour', 0.89)
('rit', 1.0)
('ure', 1.0)
('.', 0.36)
(' ', 0.1)


### We do the same with the top other candidates

In [ ]:
# Les autres suggestions
top_five = response['choices'][0]["logprobs"]["top_logprobs"]
for k in range(len(top_five)) :
  print('Suggestion n°'+str(k)+"\t", *[(i[0], round(np.exp(i[1]),3)) for i in top_five[k].items()], sep=",  ", end="\n\n")

Suggestion n°0	,  ('\n\n', 0.365),  ('\n', 0.25),  (' du', 0.012),  (' la', 0.012),  (' le', 0.01)

Suggestion n°1	,  ('de', 0.607),  ('du', 0.086),  ('des', 0.079),  ('De', 0.054),  ('...', 0.031)

Suggestion n°2	,  (' la', 0.999),  (' l', 0.0),  (' tout', 0.0),  (' nombreux', 0.0),  (' cro', 0.0)

Suggestion n°3	,  (' nour', 0.893),  (' vi', 0.098),  (' p', 0.004),  (' sour', 0.003),  (' cro', 0.001)

Suggestion n°4	,  ('rit', 1.0),  ('r', 0.0),  ('ri', 0.0),  ('iture', 0.0),  ('<|endoftext|>', 0.0)

Suggestion n°5	,  ('ure', 1.0),  ('ur', 0.0),  ('u', 0.0),  ('<|endoftext|>', 0.0),  ('ue', 0.0)

Suggestion n°6	,  (',', 0.363),  ('.', 0.356),  (' ou', 0.177),  (' comme', 0.026),  ('.\n', 0.014)

Suggestion n°7	,  ('<|endoftext|>', 0.874),  (' ', 0.098),  (' Il', 0.016),  (' C', 0.007),  (' \n', 0.002)



In [ ]:
x = str(input("prompt ?\n\n"))
response = openai.Completion.create(
    model="gpt-3.5-turbo-instruct",  # Ou "davinci-002"
    prompt=x,  # Votre texte d'entrée
    max_tokens=50,  # Nombre maximal de tokens pour la réponse
    logprobs=5,  # Demander les logprobs pour les 5 meilleurs tokens
    top_p=1,
    presence_penalty=1,
    frequency_penalty=2
)
print(response['choices'][0]["text"])

prompt ?

Le chat mange ...

de la nourriture.


In [ ]:
probs = [np.exp(i) for i in response['choices'][0]["logprobs"]["token_logprobs"]]

print(*[(j, round(probs[i],2)) for i, j in enumerate(response['choices'][0]["logprobs"]["tokens"])], sep="\n")

('\n', 0.25)
('de', 0.59)
(' la', 1.0)
(' nour', 0.99)
('rit', 1.0)
('ure', 1.0)
('.', 0.82)


## Temperature

In [14]:
x = str(input("prompt ?\n"))
t = float(input("temperature ?\n"))
response = openai.Completion.create(
    model="gpt-3.5-turbo-instruct",  # Ou "davinci-002"
    prompt=x,  # Votre texte d'entrée
    max_tokens=25,  # Nombre maximal de tokens pour la réponse
    logprobs=5,  # Demander les logprobs pour les 5 meilleurs tokens
    top_p=1,
    presence_penalty=0,
    frequency_penalty=0,
    temperature=t
)
print(response['choices'][0]["text"])
probs = [np.exp(i) for i in response['choices'][0]["logprobs"]["token_logprobs"]]
print(*[(j, round(probs[i],2)) for i, j in enumerate(response['choices'][0]["logprobs"]["tokens"])], sep="\n")

prompt ?
Le chat mange ...
temperature ?
0


de la nourriture, principalement des croquettes ou de la pâtée spécialement conçues
('\n\n', 0.37)
('de', 0.61)
(' la', 1.0)
(' nour', 0.89)
('rit', 1.0)
('ure', 1.0)
(',', 0.36)
(' princip', 0.28)
('alement', 1.0)
(' des', 0.75)
(' cro', 0.99)
('quet', 1.0)
('tes', 1.0)
(' ou', 0.86)
(' de', 0.9)
(' la', 1.0)
(' p', 0.98)
('ât', 1.0)
('ée', 0.97)
(' spéc', 0.55)
('ia', 0.99)
('lement', 1.0)
(' con', 0.97)
('ç', 1.0)
('ues', 0.71)


# Presence penalty, frequency penalty, temperature

In [ ]:
def run():
  x = str(input("prompt ?\n\n"))
  p = float(input("penalty ?\n\n"))
  f = float(input("frequency ?\n\n"))
  t = float(input("temperature ?\n\n"))
  response = openai.Completion.create(
      model="gpt-3.5-turbo-instruct",  # Ou "davinci-002"
      prompt=x,  # Votre texte d'entrée
      max_tokens=25,  # Nombre maximal de tokens pour la réponse
      logprobs=5,  # Demander les logprobs pour les 5 meilleurs tokens
      top_p=1,
      presence_penalty=p,
      frequency_penalty=f,
      temperature=t
  )
  print(response['choices'][0]["text"], end="\n")
  #probs = [np.exp(i) for i in response['choices'][0]["logprobs"]["token_logprobs"]]
  #print(*[(j, round(probs[i],2)) for i, j in enumerate(response['choices'][0]["logprobs"]["tokens"])], sep="\n")

In [ ]:
run()

prompt ?

Le chat mange ...
penalty ?

0
frequency ?

0
temperature ?

0


de la nourriture, principalement des croquettes ou de la pâtée spécialement conçues


In [ ]:
run()

prompt ?

Le chat mange ...
penalty ?

0
frequency ?

2
temperature ?

0


de la nourriture, principalement des croquettes ou de la pâtée spécialement conçues


In [ ]:
run()

prompt ?

Le chat mange ...
penalty ?

0
frequency ?

0
temperature ?

1


de la nourriture


In [ ]:
run()

prompt ?

Le chat mange ...
penalty ?

2
frequency ?

0
temperature ?

1



des croquettes.


In [ ]:
run()

prompt ?

Le chat mange ...
penalty ?

0
frequency ?

2
temperature ?

1


de la nourriture.


***
***
***
<h1>Draft</h1>

***
***
***

In [ ]:
import openai
import math
import json

# Effectuer une requête avec l'API de complétion pour obtenir les logprobs
response = openai.Completion.create(
    model="gpt-3.5-turbo-instruct",  # Ou "gpt-4" si vous y avez accès
    prompt="Le chat mange...",  # Votre texte d'entrée
    max_tokens=50,  # Nombre maximal de tokens pour la réponse
    logprobs=5,  # Demander les logprobs pour les 5 meilleurs tokens
    top_p=1,
    presence_penalty=0.5,
    frequency_penalty=0.5
)

def transformer_logprobs_en_liste(choice_logprobs):
    """
    Transforme les logprobs en une structure liste de listes.

    Args:
        choice_logprobs (list): Liste des logprobs par token.

    Returns:
        dict: Dictionnaire où chaque token est une clé et la valeur est une liste de tuples (token, probabilité).
    """
    resultat = {}
    for idx, token_logprob in enumerate(choice_logprobs):
        token_cle = f"token_{idx + 1}"
        candidats = []
        for token, logprob in token_logprob.items():
            proba = math.exp(logprob)  # Convertir logprob en probabilité
            candidats.append((token, proba))
        resultat[token_cle] = candidats
    return resultat

# Afficher la réponse générée et les logprobs transformés
for choice in response['choices']:
    print(f"Texte généré : {choice['text']}")
    if 'logprobs' in choice:
        # Transformer les logprobs
        logprobs_top = choice['logprobs']['top_logprobs']
        resultat_structure = transformer_logprobs_en_liste(logprobs_top)

        # Affichage en JSON pour lisibilité
        print("Logprobs structurés :")
        print(json.dumps(resultat_structure, indent=4, ensure_ascii=False))


Texte généré : 
Il mange de la nourriture, comme des croquettes, du poisson, ou de la viande.
Logprobs structurés :
{
    "token_1": [
        [
            "\n\n",
            0.25575165576932496
        ],
        [
            "\n",
            0.23444126301934132
        ],
        [
            " des",
            0.03496629435225372
        ],
        [
            " du",
            0.019341075120310296
        ],
        [
            " un",
            0.018130983655608286
        ]
    ],
    "token_2": [
        [
            "de",
            0.34860785958802143
        ],
        [
            "des",
            0.13337090136389648
        ],
        [
            "La",
            0.11102081197217717
        ],
        [
            "du",
            0.08739406541991676
        ],
        [
            "...",
            0.08180323405747644
        ]
    ],
    "token_3": [
        [
            " mange",
            0.8110580168520024
        ],
        [
            " n

In [ ]:
response['choices'][0]["logprobs"]

<OpenAIObject at 0x7a4ba1f1a8e0> JSON: {
  "tokens": [
    "\n",
    "Il",
    " mange",
    " de",
    " la",
    " nour",
    "rit",
    "ure",
    ",",
    " comme",
    " des",
    " cro",
    "quet",
    "tes",
    ",",
    " du",
    " po",
    "isson",
    ",",
    " ou",
    " de",
    " la",
    " vi",
    "ande",
    "."
  ],
  "token_logprobs": [
    -1.4505502,
    -3.8228374,
    -0.20941569,
    -0.43216413,
    -0.0010569837,
    -0.006264712,
    -0.00017791527,
    -0.00013882013,
    -0.7260557,
    -0.53006744,
    -0.029071022,
    -2.4153549e-05,
    -5.5577775e-06,
    -0.00020783172,
    -0.8520426,
    -1.4265171,
    -0.24791998,
    -0.0002257086,
    -2.016134,
    -1.1223005,
    -0.18872976,
    -0.00014132341,
    -0.006179171,
    -0.00073994225,
    -0.0096003665
  ],
  "top_logprobs": [
    {
      "\n\n": -1.3635484,
      "\n": -1.4505502,
      " des": -3.3533707,
      " du": -3.9455242,
      " un": -4.010133
    },
    {
      "de": -1.0538076,
  

In [ ]:
print([i for i in response['choices'][0]])

['text', 'index', 'logprobs', 'finish_reason']


In [ ]:
print(response['choices'][0]["text"])


Il mange de la nourriture, comme des croquettes, du poisson, ou de la viande.


In [ ]:
print([i for i in response['choices'][0]["logprobs"]])

['tokens', 'token_logprobs', 'top_logprobs', 'text_offset']


In [ ]:
response['choices'][0]["logprobs"]["tokens"]

['\n',
 'Il',
 ' mange',
 ' de',
 ' la',
 ' nour',
 'rit',
 'ure',
 ',',
 ' comme',
 ' des',
 ' cro',
 'quet',
 'tes',
 ',',
 ' du',
 ' po',
 'isson',
 ',',
 ' ou',
 ' de',
 ' la',
 ' vi',
 'ande',
 '.']

In [ ]:
[np.exp(i) for i in response['choices'][0]["logprobs"]["token_logprobs"]]

[0.23444126301934132,
 0.021865671137173668,
 0.8110580168520024,
 0.649102830694088,
 0.9989435747105101,
 0.9937548703941953,
 0.9998221005559831,
 0.9998611895050684,
 0.48381353730343596,
 0.588565275497703,
 0.9713474749763803,
 0.9999758467426946,
 0.9999944422379444,
 0.9997921898755158,
 0.42654278523539846,
 0.2401438644551228,
 0.7804223901791626,
 0.9997743168702697,
 0.13316930371822494,
 0.3255300506791365,
 0.8280102379495083,
 0.9998586865756827,
 0.9938398808154502,
 0.9992603314397577,
 0.9904455698988481]